# OntoBOT – Interactive Notebook

This notebook allows you to:
- Load the OntoBOT ontology, action graph, and observation graph
- Explore the ontology's structure (classes, properties)
- Query each graph separately using SPARQL

Ensure all three `.ttl` files are in the following paths:
- `../ontology/OntoBOT.ttl`
- `../kg-data/action-graph/kg.ttl`
- `../kg-data/observation-graph/kg.ttl`

Further details about the PSR - Action Ontology are available [here](https://kai-vu.github.io/OntoBOT/)
The KGs were created based on a kitchen environment built in Webots. The images of the environment were given to different multimodal LMs, which created the KGs. More work is ongoing to automise this process for more seamless integration. Images can be found in the folder `images`. 

#### Set up

In [ ]:
from rdflib import Graph, Namespace
from rdflib.namespace import RDF, RDFS, OWL
from rdflib import Graph, RDF, RDFS

import pprint

#### Load Ontology and KGs

In [ ]:
# Load the ontology
ontology_graph = Graph()
ontology_graph.parse("../ontology/OntoBOT.ttl", format="turtle")
print(f"Ontology loaded with {len(ontology_graph)} triples.")

# Load the action graph
action_graph = Graph()
action_graph.parse("../kg-data/action-graph/kg.ttl", format="turtle")
print(f"Action graph loaded with {len(action_graph)} triples.")

# Load the observation graph
obs_graph = Graph()
obs_graph.parse("../kg-data/observation-graph/kg.ttl", format="turtle")
print(f"Observation graph loaded with {len(obs_graph)} triples.")

In [ ]:
PSR = Namespace("https://w3id.org/OntoBOT#")
ontology_graph.bind("psr", PSR)
action_graph.bind("psr", PSR)
obs_graph.bind("psr", PSR)

#### Explore the Ontology

In [ ]:
# List all classes in the ontology

q_classes = """
SELECT DISTINCT ?classLabel WHERE {
  ?class a owl:Class ;
  rdfs:label ?classLabel
}
"""

results = ontology_graph.query(q_classes)
print("Ontology Classes:")
for row in results:
    pprint(row)

In [ ]:
# List all obkect properties in the ontology

q_obj_props = """
SELECT DISTINCT ?propLabel WHERE {
  ?prop a owl:ObjectProperty ;
    rdfs:label ?propLabel .
}
"""

results = ontology_graph.query(q_obj_props)
print("Object Properties:")
for row in results:
    pprint(row)

#### Query the KGs

In [ ]:
# What are the objects in the environment?

cq1 = """
PREFIX obot: <https://w3id.org/psr-action#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?obj WHERE {
  ?environment dul:hasComponent ?obj
}
"""
results = obs_graph.query(cq1)
print("Objects on the table:")
for row in results:
    pprint(row)

In [ ]:
# What are the objects on the kitchen counter? 

cq2 = """
PREFIX obot: <https://w3id.org/psr-action#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX ex: <http://example.org/data/>

SELECT DISTINCT ?obj WHERE {
  ?obj a obot:Object ;
    dul:hasLocation ?loc .
  ?loc a obot:CurrentLocation ;
    obot:onTopOf ex:Counter .
}
"""
results = obs_graph.query(cq2)
print("Objects on the table:")
for row in results:
    pprint(row)

In [ ]:
# What are the actions the robots has to perform?

cq3 = """
PREFIX obot: <https://w3id.org/psr-action#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>

SELECT ?action ?actionLabel WHERE {
  ?action a dul:Action ;
    rdfs:label ?actionLabel .
}
"""
results = action_graph.query(cq3)
print("Actions:")
for row in results:
    pprint(row)

In [ ]:
# What are the actions and their order related to the objects on the counter?

cq4 = """
PREFIX obot: <https://w3id.org/OntoBOT#>
PREFIX dul: <http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#>
PREFIX ex: <http://example.org/data/>
PREFIX example: <http://example.com/ns#> 
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT DISTINCT ?action ?actionLabel ?targetObj ?targetLabel ?follows ?followsLabel ?precedes ?precedesLabel WHERE {
  {
    ?obj a obot:Object ;
         rdfs:label ?objLabel ;
         dul:hasLocation ?loc .
    ?loc a obot:CurrentLocation ;
         obot:onTopOf ex:Counter .

    ?action a dul:Action ;
            rdfs:label ?actionLabel ;
            obot:actsOn ?targetObj .
    ?targetObj rdfs:label ?targetObjLabel .
    OPTIONAL { ?action obot:follows ?follows . ?follows rdfs:label ?followsLabel . }
    OPTIONAL { ?action obot:precedes ?precedes . ?precedes rdfs:label ?precedesLabel . }
    BIND(?objLabel AS ?targetObjLabel)
  }
  UNION
  {
    ?linkedAction a dul:Action ;
                  obot:actsOn ?linkedObj ;
                  rdfs:label ?linkedLabel .
    {
      ?linkedAction obot:follows ?action .
    } UNION {
      ?linkedAction obot:precedes ?action .
    }

    ?action a dul:Action ;
            rdfs:label ?actionLabel ;
            obot:actsOn ?targetObj .
    OPTIONAL { ?action obot:follows ?follows . ?follows rdfs:label ?followsLabel . }
    OPTIONAL { ?action obot:precedes ?precedes . ?precedes rdfs:label ?precedesLabel . }
    OPTIONAL { ?targetObj rdfs:label ?targetLabel . }
}
}
"""
combined_graph = action_graph + obs_graph
results = combined_graph.query(cq4)
print("Actions:")
for row in results:
    pprint(row)